<a href="https://colab.research.google.com/github/Summertree-8/machine_learning/blob/main/Adjust_hyper_parameter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

ハイパーパラメータ  
アルゴリズムの制御  
学習実行の前に行う  
ホールドアウト法：2分割　学習用　テスト用  
3分割　train validation test　→データ少ないと偏る→  
ｋ分割交差検証  
調整法：  
手動、グリッドサーチ、ランダム、ベイズ最適化


In [1]:
import numpy as np
import pandas as pd

In [2]:
from sklearn.datasets import load_breast_cancer

In [4]:
dataset = load_breast_cancer()#インスタンス化
dataset

{'DESCR': '.. _breast_cancer_dataset:\n\nBreast cancer wisconsin (diagnostic) dataset\n--------------------------------------------\n\n**Data Set Characteristics:**\n\n    :Number of Instances: 569\n\n    :Number of Attributes: 30 numeric, predictive attributes and the class\n\n    :Attribute Information:\n        - radius (mean of distances from center to points on the perimeter)\n        - texture (standard deviation of gray-scale values)\n        - perimeter\n        - area\n        - smoothness (local variation in radius lengths)\n        - compactness (perimeter^2 / area - 1.0)\n        - concavity (severity of concave portions of the contour)\n        - concave points (number of concave portions of the contour)\n        - symmetry \n        - fractal dimension ("coastline approximation" - 1)\n\n        The mean, standard error, and "worst" or largest (mean of the three\n        largest values) of these features were computed for each image,\n        resulting in 30 features.  For

In [7]:
x = dataset.data
t = dataset.target

In [8]:
x.shape, t.shape #データ数

((569, 30), (569,))

In [9]:
from sklearn.model_selection import train_test_split

In [11]:
x_train_val, x_test, t_train_val, t_test = train_test_split(x, t, test_size=0.2, random_state=1)#分割

In [12]:
x_train_val.shape

(455, 30)

In [13]:
x_train, x_val, t_train, t_val = train_test_split(x_train_val, t_train_val, test_size=0.3, random_state=1)#分割

In [16]:
x_train.shape, x_val.shape, x_test.shape

((318, 30), (137, 30), (114, 30))

決定木の実装

In [17]:
from sklearn.tree import DecisionTreeClassifier

In [23]:
dtree = DecisionTreeClassifier(random_state=0)

In [24]:
dtree.fit(x_train, t_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=0, splitter='best')

In [25]:
DecisionTreeClassifier(random_state=0)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=0, splitter='best')

In [26]:
print(f'train score: {dtree.score(x_train, t_train)}')
print(f'val score: {dtree.score(x_val, t_val)}')

train score: 1.0
val score: 0.927007299270073


若干過学習

In [28]:
dtree = DecisionTreeClassifier(max_depth=10, min_samples_split=30, random_state=0)#add hyper parameter

In [29]:
dtree.fit(x_train, t_train)
DecisionTreeClassifier(random_state=0)
print(f'train score: {dtree.score(x_train, t_train)}')
print(f'val score: {dtree.score(x_val, t_val)}')

train score: 0.9308176100628931
val score: 0.9562043795620438


In [30]:
print(f'test score: {dtree.score(x_test, t_test)}')

test score: 0.9298245614035088


手動はやはり効率が悪い→グリッドサーチ  
網羅できるが時間かかる可能性あり

In [31]:
from sklearn.model_selection import GridSearchCV

estimator 学習に使用するもでる  
param_grid パイパーパラメータを探索する範囲  
CV ｋの値

In [32]:
estimator = DecisionTreeClassifier(random_state=0)

In [36]:
param_grid = [{
    'max_depth':[3,20,50],
    'min_samples_split':[3,20,30]
}]

In [34]:
cv = 5

In [37]:
tuned_model = GridSearchCV(estimator=estimator,
                           param_grid=param_grid,
                           cv=cv,
                           return_train_score=False)

In [38]:
tuned_model.fit(x_train_val, t_train_val)

GridSearchCV(cv=5, error_score=nan,
             estimator=DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features=None,
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              presort='deprecated',
                                              random_state=0, splitter='best'),
             iid='deprecated', n_jobs=None,
             param_grid=[{'max_depth': [3, 20, 50],
                          'min_samples_split': [3, 20, 30]}],
             

In [ ]:
#学習結果：cv_results_  →pandasに変換して表示

In [42]:
pd.DataFrame(tuned_model.cv_results_).T

,0,1,2,3,4,5,6,7,8
mean_fit_time,0.00433264,0.00409703,0.00405569,0.00479622,0.00585876,0.00573769,0.00479269,0.00462556,0.00471678
std_fit_time,0.000444424,0.000268574,9.83991e-05,0.000237287,0.00127195,0.000994887,0.00015375,0.000352205,0.000286817
mean_score_time,0.000588942,0.000425339,0.000464344,0.000467873,0.000471973,0.000471926,0.000450563,0.000391388,0.000467873
std_score_time,0.000231217,2.27951e-05,5.10507e-05,2.18569e-05,2.46011e-05,0.000100484,4.83686e-05,2.9216e-05,8.8975e-05
param_max_depth,3,3,3,20,20,20,50,50,50
param_min_samples_split,3,20,30,3,20,30,3,20,30
params,"{'max_depth': 3, 'min_samples_split': 3}","{'max_depth': 3, 'min_samples_split': 20}","{'max_depth': 3, 'min_samples_split': 30}","{'max_depth': 20, 'min_samples_split': 3}","{'max_depth': 20, 'min_samples_split': 20}","{'max_depth': 20, 'min_samples_split': 30}","{'max_depth': 50, 'min_samples_split': 3}","{'max_depth': 50, 'min_samples_split': 20}","{'max_depth': 50, 'min_samples_split': 30}"
split0_test_score,0.923077,0.912088,0.912088,0.956044,0.912088,0.912088,0.956044,0.912088,0.912088
split1_test_score,0.901099,0.901099,0.901099,0.912088,0.901099,0.901099,0.912088,0.901099,0.901099
split2_test_score,0.934066,0.934066,0.934066,0.923077,0.934066,0.934066,0.923077,0.934066,0.934066


mean_fit_time　学習時間平均  
std_fit_time　学習時間標準偏差  
mean_score_time　検証時間平均  
std_score_time　検証時間標準偏差  
param_max_depth	
param_min_samples_split	
params  
split0_test_score  
split1_test_score	 
split2_test_score  
split3_test_score	  
split4_test_score  
mean_test_score　検証用データに対する予測精度の平均	
std_test_score	検証用データに対する予測精度の標準偏差	
rank_test_score　検証用データに対する予測精度のランク  

・３のハイパーパラメータの組み合わせの精度が一番よさそう
→さっきより狭い範囲で調整する


In [43]:
param_grid = [{
    'max_depth':[5,10,15],
    'min_samples_split':[10,12,15]
}]
cv = 5
tuned_model = GridSearchCV(estimator=estimator,
                           param_grid=param_grid,
                           cv=cv,
                           return_train_score=False)
tuned_model.fit(x_train_val, t_train_val)
pd.DataFrame(tuned_model.cv_results_).T

,0,1,2,3,4,5,6,7,8
mean_fit_time,0.00518703,0.00460649,0.00481563,0.00461802,0.00475297,0.00458298,0.00471101,0.00467267,0.00466342
std_fit_time,0.00108244,0.000224448,0.000216894,0.000192644,0.000387937,0.000259372,0.000229986,0.000251974,0.000156175
mean_score_time,0.000456476,0.000399065,0.000500107,0.000395489,0.000432396,0.000400114,0.000484848,0.000420952,0.000503492
std_score_time,5.49549e-05,2.44833e-05,7.46948e-05,1.50732e-05,3.00133e-05,5.9047e-06,9.81039e-05,1.46262e-05,0.00011016
param_max_depth,5,5,5,10,10,10,15,15,15
param_min_samples_split,10,12,15,10,12,15,10,12,15
params,"{'max_depth': 5, 'min_samples_split': 10}","{'max_depth': 5, 'min_samples_split': 12}","{'max_depth': 5, 'min_samples_split': 15}","{'max_depth': 10, 'min_samples_split': 10}","{'max_depth': 10, 'min_samples_split': 12}","{'max_depth': 10, 'min_samples_split': 15}","{'max_depth': 15, 'min_samples_split': 10}","{'max_depth': 15, 'min_samples_split': 12}","{'max_depth': 15, 'min_samples_split': 15}"
split0_test_score,0.967033,0.923077,0.912088,0.967033,0.923077,0.912088,0.967033,0.923077,0.912088
split1_test_score,0.912088,0.901099,0.901099,0.912088,0.901099,0.901099,0.912088,0.901099,0.901099
split2_test_score,0.923077,0.934066,0.934066,0.923077,0.934066,0.934066,0.923077,0.934066,0.934066


さっきより高い値出ている↑

In [44]:
tuned_model.best_params_

{'max_depth': 5, 'min_samples_split': 10}

In [46]:
best_model = tuned_model.best_estimator_

In [47]:
print(best_model.score(x_train_val,t_train_val))
print(best_model.score(x_test,t_test))

0.9934065934065934
0.956140350877193
